# OpenAI Quickstart

# Overview  
>"Large Language Model(LLM)은 텍스트를 텍스트에 매핑하는 기능입니다. 입력 문자열이 주어지면 LLM은 다음에 나올 텍스트를 예측하려고 합니다"(1). 이 "QuickStart" 노트북은 사용자에게 고급 LLM 개념, AML을 시작하기 위한 핵심 패키지 요구 사항, 프롬프트 디자인에 대한 간략한 소개 및 다양한 사용 사례에 대한 몇 가지 간단한 예를 소개합니다.

더 많은 빠른 시작 예제를 보려면 [공식 Azure Open AI 빠른 시작 설명서](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio)를 참조하세요.

>해당 컨텐츠는 DevContainer 기반으로 개발에 필요한 환경설정이 정의되어져 있습니다. GitHub Codespace를 활용하거나, 로컬에 Docker를 설치한 상태에서 Visual Stduio Code IDE에 해당 Repository를 다운로드 받을 경우, 자동으로 컨테이너에 개발환경(Python Runtime 3.11.4, Azure OpenAI 1.13.3)을 설치합니다. .env 파일에 필수 API 정보를 입력하고 저장후 사용하세요.

## Table of Contents  

[Overview](#overview)  
[How to use OpenAI Service](#how-to-use-openai-service)  
[1. Creating your OpenAI Service](#1.-creating-your-openai-service)  
[2. Installation](#2.-installation)    
[3. Credentials](#3.-credentials)  

[Use Cases](#use-cases)    
[1. Summarize Text](#1.-summarize-text)  
[2. Classify Text](#2.-classify-text)  
[3. Generate New Product Names](#3.-generate-new-product-names)  
[4. Fine Tune a Classifier](#4.fine-tune-a-classifier)  
[5. Embeddings!]((#5.-embeddings!))

[References](#references)

### Getting started with Azure OpenAI Service

신규 고객은 Azure OpenAi 서비스에 [https://aka.ms/oai/access) [액세스 신청] (https://aka.ms/oai/access)해야합니다.
승인이 완료된 후 고객은 Azure Portal에 로그인하고 Azure OpenAI 서비스 리소스를 만들고 스튜디오를 통해 모델 실험을 시작할 수 있습니다.

[Great resource for getting started quickly](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177 )


### Build your first prompt  
이 짧은 연습은 간단한 작업 "요약"을 위해 OpenAI 모델에 프롬프트를 제출하기위한 기본 소개를 제공합니다.

![](images/generative-AI-models-reduced.jpg)  


**Steps**:  
1. 파이썬 환경에 OpenAI 라이브러리를 설치
2. 표준 헬퍼 라이브러리를 로드하고 OpenAI 보안 자격 증명을 설정
3. OpenAI 작업에 적합한 모델을 선택
4. 모델에 대한 간단한 프롬프트를 만듭니다.
5. 모델 API에 요청을 제출하십시오!

### 1. 파이썬 환경에 OpenAI 라이브러리를 설치하십시오.
DevContainer 가 시작할 때, 자동으로 `requirements.txt` 에 기술한 라이브러리를 설치합니다.
따라서 추가적으로 라이브러리 설치를 진행하지 않아도 즉시 실습할 수 있습니다.
Python 버전은 **Python 3.11.4**, **Azure OpenAI 1.13.3** 를 사용합니다.

### 2. 표준 헬퍼 라이브러리를 로드하고 OpenAI 보안 자격 증명을 설정
루트 디렉토리에 존재하는 .evn.sample 파일을 복사하여 .env 파일을 생성하고 Azure OpenAI Endpoint URL(AZURE_OPENAI_ENDPOINT) 및 API Key(AZURE_OPENAI_API_KEY)를 넣습니다.  
만약, 정상 하단 코드 실행이 정상적으로 되지 않을 경우, 해당 파일을 저장 후 파일을 닫은 후 새로 열어서 진행합니다. (커널 다시 시작을 해도 됩니다.)

In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

### 3. 작업에 적합한 OpenAI 모델 선택
gpt-35-turbo / gpt-4 또는 text-embedding-ada-002 와 같은 OpenAI 모델을 선택합니다.  
  
2024년 3월 기준, 본 실습에서 사용하는 LLM 모델은 아래와 같이 크게 3종류로 나뉘어집니다.  

* gpt-35-turbo / gpt-35-turbo-16k
* gpt-4-turbo / gpt-4 / gpt-4-32k
* text-embedding-ada-002 / text-embedding-3-small / text-embedding-3-large

주로 gpt-35-turbo 모델을 활용합니다. 한글의 토큰 길이에 대한 제약이 있을 경우에 gpt-35-turbo-16k 모델을 활용할 수 있습니다. (gpt-35-turbo-0125 버전은 16K 까지 프롬프트를 활용할 수 있습니다.) 
좀 더 복합한 작업을 처리할 때에는 gpt-4-turbo 모델이 유용할 수 있습니다.  
  
모델에 자세한 정보는 다음을 참고하세요: [Azure OpenAI models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  

In [2]:
# Select the General Purpose gpt-35-turbo model for text
# deployment_name = os.getenv("DEPLOYMENT_NAME")
deployment_name = "gpt-35-turbo"

## 4. Prompt Design  

대형 언어 모델(LLM)의 마법은 방대한 양의 텍스트 예측 오류를 최소화 하도록 훈련함으로써 모델이 결국 유용하게 예측하도록 개념을 학습하는 것입니다.  
예를 들어 다음과 같은 개념을 학습합니다.(1):

* 어떻게 쓰는지
* 문법의 작동 방식
* 의역하는 방법
* 질문에 대답하는 방법
* 대화를 이끄는 방법
* 여러 언어로 작성하는 방법
* 코딩하는 방법
* 기타

### 대형 언어 모델(LLM)을 제어하는 방법  
LLM에 대한 모든 입력 중에서 가장 영향력 있는 것은 **텍스트 프롬프트(1)**입니다.

대규모 언어 모델은 몇 가지 방법으로 출력을 생성하라는 메시지를 표시할 수 있습니다: 

* 지침(Instruction): 모델에게 원하는 것을 말하십시오.
* 완료(Completion): 모델이 원하는 것의 시작을 끝낼수 있도록 유도하십시오.
* 시연(Demonstration): 다음 중 하나를 사용하여 원하는 것을 모델에 보여주십시오.  
프롬프트 내에 몇 가지 예시를 주거나, 또는 수백 또는 수천 개의 예로 구성된 훈련 데이터 세트를 이용한 미세 조정


#### 프롬프트를 만들기 위한 세 가지 기본 지침

* **Show and tell**: 지침이나 예시 또는 두 가지 조합을 통해 원하는 것을 명확히 하십시오. 모델이 알파벳 순서로 항목 목록을 순위에 올리거나 정서적으로 단락을 분류하려면 원하는 것임을 보여주십시오.

* **Provide quality data**: 분류기를 구축하거나 모델이 패턴을 따를 경우 충분한 예시가 있는지 확인하십시오. 예제를 교정하십시오. 모델은 일반적으로 기본 철자 실수를 통해 보고 응답을 제공 할 수있을 정도로 똑똑하지만 의도적이며 응답에 영향을 줄 수 있다고 가정 할 수도 있습니다.

* **Check your setting**: 온도(Temperature) 및 TOP_P 설정은 모델이 응답을 생성하는 데 결정적인 방법을 제어합니다. 정답이 하나만 있는 응답을 요청하는 경우 온도를 더 낮게 설정하고 싶을 것입니다. 더 다양한 응답을 찾고 있다면 더 높은 응답을 원할 수도 있습니다. 사람들이 이러한 설정에서 사용하는 가장 큰 실수는 그들이 "영리" 또는 "창의성" 컨트롤을 가정합니다.  

Source: https://github.com/Azure/OpenAI/blob/main/How%20to/Completions.md

### 5. Submit!

In [3]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

In [4]:
# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    max_tokens=60,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": text_prompt},
    ]
)

print(json.dumps(response.model_dump(), indent=2))

{
  "id": "chatcmpl-91e5cXw7TXVzynr80hYInsm1aWGKp",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The use of Oxford commas, also known as serial commas, is a matter of style and preference in writing. Some style guides, such as the Chicago Manual of Style, recommend using the Oxford comma consistently to avoid ambiguity. Others, such as the Associated Press Stylebook, suggest using it only when necessary",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }

### 동일한 호출을 반복합니다. 결과는 어떻게 비교됩니까?

In [5]:
# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    max_tokens=60,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": text_prompt},
    ]
)

print(json.dumps(response.model_dump(), indent=2))

{
  "id": "chatcmpl-91e5ddThAmXaB5qrUjWC0LWm99P7n",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The use of Oxford commas, also known as serial commas, is a matter of style and preference. Some style guides, including the Chicago Manual of Style and the Oxford Style Manual, recommend using the Oxford comma consistently. It is generally used before the conjunction in a series of three or more items, such",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }

# Exercises for several use cases  
1. Summarize Text  
2. Classify Text  
3. Generate New Product Names
4. Embeddings
5. Fine tune a classifier

## Summarize Text  
LLM은 다양한 케이스에서 사용이 가능합니다. 요약하기 위한 방법은 다음과 같습니다.
결과물에 대한 확인 방법:
1. 토크나이저: https://platform.openai.com/tokenizer 

In [6]:
prompt = """
한화생명(대표이사 여승주 부회장)은 올해 첫 신상품으로 출시된 '한화생명 The H 건강보험'이 출시 43일(평일 기준) 만에 누적 판매건수 10만건을 달성했다고 7일 밝혔다.
이 상품은 생명보험업계 최초로 뇌·심장에 새로운 위험률을 적용해 보험료를 50~60%가량 절감한 것이 특징이다.
한화생명은 해당 상품이 매일 2500건 가까이 판매되면서 대표 상품으로 자리매김하고 있다고 설명했다.
The H 건강보험이 높은 판매고를 기록하면서 한화생명의 지난 1월 총 신계약 건수는 11만5000건을 달성했다. 지난 2021년 4월 이후 최초로 월별 신계약 건수가 10만건을 돌파했다.
구창희 한화생명 일반보장팀 팀장은 “이 상품이 고객과 FP의 사랑을 받아 단시간에 10만건을 달성할 수 있었던 건 뇌·심장 신 위험률을 타 생보사보다 빠르게 반영해 경쟁력을 확보한 결과”라며 “앞으로도 설계사가 고객에게 자신 있게 제안할 수 있는 다양한 상품을 선제적으로 선보이기 위해 최선을 다하겠다”고 말했다.
"""

deployment_name = "gpt-35-turbo"

In [7]:
#Setting a few additional, typical parameters during API Call
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": "You are a summarise assistant. Summarize the following text using 20 words."},
    {"role": "user", "content": prompt},
  ],
  temperature=0.7,
  max_tokens=200
)

print(response.choices[0].message.content)
# 공백 개수 세기
print(str(response.choices[0].message.content.count(" ")) + "개")

한화생명이 출시한 '한화생명 The H 건강보험'이 출시 43일 만에 10만건을 판매하며 성공적인 출시를 이루었다. 이는 생명보험업계에서 처음으로 뇌·심장 위험률을 적용한 것이다.
20개


## Classify Text  
#### Challenge  
추론 시간에 제공된 범주로 항목을 분류하십시오. 다음 예에서는 프롬프트에서 분류 할 범주와 텍스트를 모두 제공합니다 (*Playground_reference).

고객 문의 : 안녕하세요, 최근 노트북 키보드의 열쇠 중 하나가 최근에 파산되었으며 교체가 필요합니다.

분류 카테고리 :

In [8]:
prompt = """Classify the following inquiry into one of the following:

categories: [Pricing, Hardware Support, Software Support]

inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement.

Classified category:
"""
deployment_name = "gpt-35-turbo"

In [9]:
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": "You are a helpful  assistant."},
    {"role": "user", "content": prompt},
  ],
  temperature=0,
  max_tokens=60
)

print(response.choices[0].message.content)

Hardware Support


## Generate New Product Names
#### Challenge
예제 단어에서 제품 이름을 만듭니다.여기에는 이름을 생성 할 제품에 대한 프롬프트 정보가 포함되어 있습니다.우리는 또한 우리가 받고자하는 패턴을 보여주는 비슷한 예를 제공합니다.또한 임의성과보다 혁신적인 반응을 높이기 위해 온도 값을 높게 설정했습니다.

제품 설명 : 홈 밀크 쉐이크 제조업체
종자 단어 : 빠르고 건강하며 소형.
제품 이름 : Homeshaker, Fit Shaker, Quickshake, Shake Maker

제품 설명 : 발 크기에 맞는 신발 한 쌍.
종자 단어 : 적응성, 적합, 옴니 피트.

In [10]:
prompt = """Product description: A home milkshake maker
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.
"""
model = "gpt-35-turbo"

In [11]:
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": "You are a helpful  assistant."},
    {"role": "user", "content": prompt},
  ],
  temperature=0.8,
  max_tokens=60
)

print(response.choices[0].message.content)

Product names: FlexFit, OmniFit, UniversalFit, AdaptaShoes, CustomFit


## Embeddings!  
이 섹션에서는 임베딩을 검색하고 단어, 문장 및 문서 사이의 유사성을 찾는 방법을 보여줍니다.

In [12]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [13]:
text = 'the quick brown fox jumped over the lazy dog'

deployment_name = 'text-embedding-ada-002'

In [14]:
vector = client.embeddings.create(input = [text], model=deployment_name).data[0].embedding
print(vector)
len(vector)

[-0.004474656656384468, 0.00978652760386467, -0.014904950745403767, -0.006424985360354185, -0.01135313231498003, 0.015513833612203598, -0.02372107096016407, -0.016414472833275795, -0.0158182755112648, -0.029632311314344406, 0.021298224106431007, 0.021095262840390205, 0.018570933490991592, 0.004170214757323265, -0.0007155169150792062, -0.007579326163977385, 0.02521790750324726, -0.004214612767100334, 0.011175542138516903, -0.008587788790464401, -0.009513798169791698, 0.021577294915914536, -0.005993693135678768, -0.008257976733148098, 0.006041261833161116, 0.013040246441960335, 0.007439790293574333, -0.0035169341135770082, -0.008955655619502068, 0.0011939817341044545, 0.00666600139811635, 0.0038657733239233494, -0.039272960275411606, -0.002559211803600192, -0.012761174701154232, -0.0217422004789114, -0.0037072100676596165, -0.010458835400640965, 0.02597901225090027, -0.0456916019320488, 0.009399632923305035, 0.015653369948267937, -0.02261747047305107, -0.01161951944231987, -0.00285731069

1536

In [15]:
sentences1 = ['The new movie is awesome',  
              'The new movie is awesome',  
              'The new movie is awesome']  
  
sentences2 = ['The dog plays in the garden',  
              'This recent movie is so good',  
              'The new movie is awesome']  

embeddings1 = [client.embeddings.create(input = s, model=deployment_name).data[0].embedding for s in sentences1]  
embeddings2 = [client.embeddings.create(input = s, model=deployment_name).data[0].embedding for s in sentences2]  
  
for i in range(len(sentences1)):  
    print("{}\t{}\tScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_similarity(embeddings1[i], embeddings2[i])))

The new movie is awesome	The dog plays in the garden	Score: 0.7488
The new movie is awesome	This recent movie is so good	Score: 0.9191
The new movie is awesome	The new movie is awesome	Score: 1.0000


In [16]:
sentences1 = ['이 시리얼은 우유와 먹으면 맛있습니다.',  
              '이 시리얼은 우유와 먹으면 맛있습니다.',
              '이 시리얼은 우유와 먹으면 맛있습니다.',  
              '이 시리얼은 우유와 먹으면 맛있습니다.',  
              '이 시리얼은 우유와 먹으면 맛있습니다.']  
  
sentences2 = ['이 시리얼은 우유와 먹으면 맛있습니다.',
              '이 씨리얼은 우유와 먹으면 맛있습니다.',
              '속초는 온천과 먹을거리를 즐길 수 있습니다.',
              '속초는 온천과 경치를 즐길 수 있습니다.',
              '자동차 보험은 언제나 캐롯 퍼마일 입니다']

embeddings1 = [client.embeddings.create(input = s, model=deployment_name).data[0].embedding for s in sentences1]  
embeddings2 = [client.embeddings.create(input = s, model=deployment_name).data[0].embedding for s in sentences2]  
  
for i in range(len(sentences1)):  
    print("{}\t{}\tScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_similarity(embeddings1[i], embeddings2[i])))

이 시리얼은 우유와 먹으면 맛있습니다.	이 시리얼은 우유와 먹으면 맛있습니다.	Score: 1.0000
이 시리얼은 우유와 먹으면 맛있습니다.	이 씨리얼은 우유와 먹으면 맛있습니다.	Score: 0.9714
이 시리얼은 우유와 먹으면 맛있습니다.	속초는 온천과 먹을거리를 즐길 수 있습니다.	Score: 0.8518
이 시리얼은 우유와 먹으면 맛있습니다.	속초는 온천과 경치를 즐길 수 있습니다.	Score: 0.8211
이 시리얼은 우유와 먹으면 맛있습니다.	자동차 보험은 언제나 캐롯 퍼마일 입니다	Score: 0.7829


# 참조  
- Azure Reference Documentation  
- Azure OpenAI GitHub Repo cookbooks  
- OpenAI website  
1 - [Openai Cookbook](https://github.com/openai/openai-cookbook)  
2 - [Azure Documentation - Azure Open AI Models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
3 - [OpenAI Studio Examples](https://oai.azure.com/portal)  
4 - [[PUBLIC] Best practices for fine-tuning GPT-3 to classify text](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#)

# 궁금증 있을 때 연락 주세요.  
Prompt Engineering과 관련 궁금증이 있을 경우 아래 이메일로 요청 주세요.  
MS Korea 김현수 이메일: [<hyounsookim@microsoft.com>](hyounsookim@microsoft.com)